In [1]:
from eo_flood_ops.model_utils import GroundTruthMeasurement, LaplaceDepthSolverConfig
from eo_flood_ops.thresholding_model import ThresholdingModel 
from eo_flood_ops.manifold_model import ManifoldModel
from eo_flood_ops.general_utils import tif_to_clipped_masked_array, tif_to_clipped_array, find_closest_valid


class MyGroundTruthMeasurement(GroundTruthMeasurement):
  pass

import hydromt  # noqa: E402
import xarray as xr  # noqa: E402
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
# Directories and filenames
images_dir = R"p:\11211461-010--eo-flood-ops\HoiAn\classified_events"
aoi_dir = R"p:\11211461-010--eo-flood-ops\HoiAn\AOI"
waterlevels_fn = R"p:\11211461-010--eo-flood-ops\HoiAn\water_levels\Water_Level.csv"
dem_fn = R"p:\11211461-010--eo-flood-ops\HoiAn\DEMs\hoi_an_FABDEM.tif"

# Load water level data
df_water = pd.read_csv(waterlevels_fn, skiprows=[1])
df_water.rename(columns={df_water.columns[0]: "datetime"}, inplace=True)
df_water["datetime"] = pd.to_datetime(df_water["datetime"])

In [3]:
results_per_aoi = {}

# Loop over AOI files
for aoi_file in os.listdir(aoi_dir):
    if not aoi_file.endswith('.geojson'):
        continue
    
    aoi_name = aoi_file.split('_')[1].split('.')[0]

    print(f"Processing AOI: {aoi_name}")
    print("*" * 45)
    # Store image-water level tuples
    image_water_levels = []
    
    # Loop over images
    for image_name in os.listdir(images_dir):
        if not image_name.endswith(".tif"):
            continue
        

        datetime_str = image_name.split("_")[1].replace(".tif", "") 
        dt = datetime.strptime(datetime_str, "%Y%m%dT%H%M%S")
        dt = pd.Timestamp(dt)
        dt_gmt7 = dt + pd.Timedelta(hours=7)  # Convert to GMT+7

        closest_time, value = find_closest_valid(df_water, dt_gmt7, aoi_name)

        if closest_time is not None:
            print(f"Image: {image_name}")
            print(f"Image timestamp (GMT+7): {dt_gmt7}")
            print("Closest valid timestamp:", closest_time)
            print(f"{aoi_name} value:", value)
            print("=" * 45)
            image_water_levels.append((image_name, value))
        else:
            print(f"No valid {aoi_name} values found in dataset for image {image_name}.")

    if not image_water_levels:
        print(f"No images found for AOI: {aoi_name}")
        continue

    # print some spacing to improve readability
    print("\n" * 1)

    # Sort by water level ascending
    image_water_levels.sort(key=lambda x: x[1])
    
    # Store results
    results_per_aoi[aoi_name] = image_water_levels

Processing AOI: ID6
*********************************************
Image: hoian_20221015T105600.tif
Image timestamp (GMT+7): 2022-10-15 17:56:00
Closest valid timestamp: 2022-10-15 18:00:00
ID6 value: 3.25
Image: hoian_20221202T105600.tif
Image timestamp (GMT+7): 2022-12-02 17:56:00
Closest valid timestamp: 2022-12-02 18:00:00
ID6 value: 1.29
Image: hoian_20221206T223600.tif
Image timestamp (GMT+7): 2022-12-07 05:36:00
Closest valid timestamp: 2022-12-07 05:40:00
ID6 value: 1.31
Image: hoian_20231115T105600.tif
Image timestamp (GMT+7): 2023-11-15 17:56:00
Closest valid timestamp: 2023-11-15 18:00:00
ID6 value: 2.06




In [4]:
# Prepare GROUND TRUTH for each AOI

# Dictionary to store GROUND_TRUTH per AOI
GROUND_TRUTH_per_aoi = {}

for aoi_file in os.listdir(aoi_dir):
    if not aoi_file.endswith('.geojson'):
        continue
    
    aoi_name = aoi_file.split('_')[1].split('.')[0]
    if aoi_name == "Modrica":  # skip for now
        continue
    
    aoi_fn = os.path.join(aoi_dir, aoi_file)
    
    # Make sure we have results for this AOI
    if aoi_name not in results_per_aoi:
        print(f"No images found for AOI: {aoi_name}")
        continue
    
    GROUND_TRUTH = []
    
    for image_name, water_level in results_per_aoi[aoi_name]:
        image_path = os.path.join(images_dir, image_name)
        
        masked_array, transforms, crs = tif_to_clipped_masked_array(
            image_path,
            aoi_fn,
        )
        
        GROUND_TRUTH.append(MyGroundTruthMeasurement(
            ground_truth=masked_array,
            gauge_measurement=water_level
        ))
    
    # Store per AOI
    GROUND_TRUTH_per_aoi[aoi_name] = GROUND_TRUTH

In [5]:
MIN_RATIOS = [0.1, 0.3, 0.5, 1, 2, 5, 10, 15, 20]
# MIN_RATIOS = [0.5, 1]

trained_models_tm= {}

for aoi_name, GROUND_TRUTH in GROUND_TRUTH_per_aoi.items():
    print("\n" + "="*60)
    print(f"Starting training for AOI: {aoi_name}")
    print(f"Number of ground truth images: {len(GROUND_TRUTH)}")
    print(f"Using minimum ratios: {MIN_RATIOS}")
    
    tm = ThresholdingModel()
    
    tm.train(
        minumum_ratios=MIN_RATIOS,
        ground_truth=GROUND_TRUTH
    )
    
    trained_models_tm[aoi_name] = tm
    print(f"Finished training for AOI: {aoi_name}")
    print("="*60)


Starting training for AOI: ID6
Number of ground truth images: 4
Using minimum ratios: [0.1, 0.3, 0.5, 1, 2, 5, 10, 15, 20]


C:\Users\tsiokano\git\EO-Flood-Ops\src\eo_flood_ops\model_utils.py:251: RuntimeWarning: divide by zero encountered in divide
  ratios = true_wets / false_wets
C:\Users\tsiokano\git\EO-Flood-Ops\src\eo_flood_ops\model_utils.py:251: RuntimeWarning: invalid value encountered in divide
  ratios = true_wets / false_wets


For min_ratio=0.1 we get f1=0.9578507578932621
For min_ratio=0.3 we get f1=0.9578507578932621
For min_ratio=0.5 we get f1=0.9615551717575752
For min_ratio=1 we get f1=0.9651129127404915
For min_ratio=2 we get f1=0.9632828107052388
For min_ratio=5 we get f1=0.957188485187689
For min_ratio=10 we get f1=0.957188485187689
For min_ratio=15 we get f1=0.957188485187689
For min_ratio=20 we get f1=0.957188485187689
chosen min_ratio 1
Finished training for AOI: ID6


In [11]:
trained_models_tm["ID6"].save(
    filedir=R"p:\11211461-010--eo-flood-ops\HoiAn\FEWS",
    filename="thresholding_model_ID6_new.pkl"
)

'p:\\11211461-010--eo-flood-ops\\HoiAn\\FEWS\\thresholding_model_ID6_new.pkl'

In [5]:
DEM_per_aoi = {}

for aoi_file in os.listdir(aoi_dir):
    if not aoi_file.endswith('.geojson'):
        continue
    if aoi_file.split('_')[1].split('.')[0] == "Modrica":  # skip for now
        continue
    
    aoi_name = aoi_file.split('_')[1].split('.')[0]
    aoi_fn = os.path.join(aoi_dir, aoi_file)
    
    # Clip DEM for this AOI
    DEM, dem_transform, dem_crs = tif_to_clipped_array(dem_fn, aoi_fn)
    
    # Store in a dictionary
    DEM_per_aoi[aoi_name] = {
        'DEM': DEM,
        'transform': dem_transform,
        'crs': dem_crs
    }

In [6]:
trained_models_mm = {}
MIN_RATIOS = [0.1, 0.3, 0.5, 1, 2, 5, 10, 15, 20]

for aoi_name, GROUND_TRUTH in GROUND_TRUTH_per_aoi.items():
    print("\n" + "="*60)
    print(f"Starting training for AOI: {aoi_name}")
    print(f"Using minimum ratios: {MIN_RATIOS}")

    # Get DEM for this AOI
    DEM = DEM_per_aoi[aoi_name]['DEM']

    print(f"Number of ground truth images: {len(GROUND_TRUTH)}")

    # Create the model
    mm = ManifoldModel(
        dem=DEM,
        scale=30,
        laplace_config=LaplaceDepthSolverConfig(
            down_scale_factor=32,
            solve_iterations_factor=3.,
            force_coeff=0.9,
            drop_iterations=1,
            drop_coeff=0.00003
        ),
        force_tolerance=1,
        force_local_region_width=5,
        flood_agree_threshold=0.1
    )

    mm.train(
        minumum_ratios=MIN_RATIOS,
        ground_truth=GROUND_TRUTH
    )

    trained_models_mm[aoi_name] = mm
    print(f"Finished training for AOI: {aoi_name}")
    print("="*60)


Starting training for AOI: ID6
Using minimum ratios: [0.1, 0.3, 0.5, 1, 2, 5, 10, 15, 20]
Number of ground truth images: 4
Training an inner thresholding model used for flood-fill.


C:\Users\tsiokano\git\EO-Flood-Ops\src\eo_flood_ops\model_utils.py:251: RuntimeWarning: divide by zero encountered in divide
  ratios = true_wets / false_wets
C:\Users\tsiokano\git\EO-Flood-Ops\src\eo_flood_ops\model_utils.py:251: RuntimeWarning: invalid value encountered in divide
  ratios = true_wets / false_wets


For min_ratio=0.1 we get f1=0.9578507578932621
For min_ratio=0.3 we get f1=0.9578507578932621
For min_ratio=0.5 we get f1=0.9615551717575752
For min_ratio=1 we get f1=0.9651129127404915
For min_ratio=2 we get f1=0.9632828107052388
For min_ratio=5 we get f1=0.957188485187689
For min_ratio=10 we get f1=0.957188485187689
For min_ratio=15 we get f1=0.957188485187689
For min_ratio=20 we get f1=0.957188485187689
chosen min_ratio 1
Running flood extent to depth on ground truth examples..
Running flood extent to depth algorithm for image at gauge_level 1.29
Running flood extent to depth algorithm for image at gauge_level 1.31
Running flood extent to depth algorithm for image at gauge_level 2.06
Running flood extent to depth algorithm for image at gauge_level 3.25
Finished training for AOI: ID6


In [7]:
trained_models_mm["ID6"].save(
    filedir=R"p:\11211461-010--eo-flood-ops\HoiAn\FEWS",
    filename="manifold_model_ID6_new.pkl"
)

'p:\\11211461-010--eo-flood-ops\\HoiAn\\FEWS\\manifold_model_ID6_new.pkl'